# Predict Tweet Toxicity

In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
# filename = "twitter_data/andrea_scraped/wikileaks2016"
filename1 = "twitter_data/andrea_scraped/abortion2016"
filename2 = "twitter_data/andrea_scraped/prochoice2016"
filename3 = "twitter_data/andrea_scraped/prolife2016"
filename4 = "twitter_data/andrea_scraped/abortion2017"
filename5 = "twitter_data/andrea_scraped/abortion2018"

In [3]:
# df = pd.read_csv(filename, encoding = "ISO-8859-1",
#                  names = ['date', 'fullname', 'id', 'likes', 'replies', 
#                             'retweets', 'text', 'url', 'user', 'month'],
#                 engine='python')

df1 = pd.read_csv(filename1, encoding = "ISO-8859-1",
                 names = ['date', 'fullname', 'id', 'likes', 'replies', 
                            'retweets', 'text', 'url', 'user', 'month'])

df2 = pd.read_csv(filename2, encoding = "ISO-8859-1",
                 names = ['date', 'fullname', 'id', 'likes', 'replies', 
                            'retweets', 'text', 'url', 'user', 'month'])

df3 = pd.read_csv(filename3, encoding = "ISO-8859-1",
                 names = ['date', 'fullname', 'id', 'likes', 'replies', 
                            'retweets', 'text', 'url', 'user', 'month'])

df4 = pd.read_csv(filename4, encoding = "ISO-8859-1",
                 names = ['date', 'fullname', 'id', 'likes', 'replies', 
                            'retweets', 'text', 'url', 'user', 'month'])

df5 = pd.read_csv(filename5, encoding = "ISO-8859-1",
                 names = ['date', 'fullname', 'id', 'likes', 'replies', 
                            'retweets', 'text', 'url', 'user', 'month'])

df = pd.concat([df1, df2, df3, df4, df5])
df = df.drop_duplicates()
df = df.dropna()
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [4]:
for date in df.sort_values('date').date.unique():
    print(date)

2016-01-01 20:54:31
2016-01-01 21:34:29
2016-01-01 21:34:31
2016-01-01 21:48:34
2016-01-01 21:54:39
2016-01-01 22:29:31
2016-01-01 22:53:29
2016-01-01 22:54:00
2016-01-01 22:55:26
2016-01-01 23:02:13
2016-01-01 23:03:12
2016-01-01 23:07:40
2016-01-01 23:08:36
2016-01-01 23:12:13
2016-01-01 23:25:10
2016-01-01 23:30:50
2016-01-01 23:37:47
2016-01-01 23:37:49
2016-01-01 23:38:23
2016-01-01 23:40:33
2016-01-01 23:49:36
2016-01-01 23:49:55
2016-01-01 23:50:53
2016-01-01 23:52:41
2016-01-01 23:52:48
2016-01-01 23:54:05
2016-01-01 23:54:43
2016-01-01 23:55:10
2016-01-01 23:56:05
2016-01-01 23:56:08
2016-01-01 23:56:21
2016-01-01 23:56:51
2016-01-01 23:57:08
2016-01-01 23:57:10
2016-01-01 23:57:22
2016-01-01 23:57:31
2016-01-01 23:57:32
2016-01-01 23:57:56
2016-01-01 23:58:43
2016-01-01 23:59:39
2016-01-01 23:59:43
2016-01-01 23:59:57
2016-01-02 23:03:01
2016-01-02 23:03:59
2016-01-02 23:04:14
2016-01-02 23:05:08
2016-01-02 23:10:10
2016-01-02 23:10:58
2016-01-02 23:12:09
2016-01-02 23:15:35


2016-01-15 23:40:08
2016-01-15 23:41:43
2016-01-15 23:45:04
2016-01-15 23:51:12
2016-01-15 23:51:17
2016-01-15 23:53:13
2016-01-15 23:53:34
2016-01-15 23:53:56
2016-01-15 23:53:59
2016-01-15 23:54:06
2016-01-15 23:54:07
2016-01-15 23:54:13
2016-01-15 23:55:02
2016-01-15 23:55:24
2016-01-15 23:55:35
2016-01-15 23:55:44
2016-01-15 23:57:12
2016-01-15 23:57:19
2016-01-15 23:57:22
2016-01-15 23:57:26
2016-01-15 23:58:06
2016-01-15 23:58:26
2016-01-15 23:58:33
2016-01-15 23:58:36
2016-01-15 23:58:52
2016-01-15 23:59:03
2016-01-15 23:59:21
2016-01-15 23:59:23
2016-01-15 23:59:44
2016-01-16 21:38:01
2016-01-16 21:38:04
2016-01-16 22:11:19
2016-01-16 22:20:05
2016-01-16 22:20:06
2016-01-16 22:31:33
2016-01-16 22:52:21
2016-01-16 22:54:01
2016-01-16 23:12:23
2016-01-16 23:17:50
2016-01-16 23:20:13
2016-01-16 23:21:46
2016-01-16 23:22:00
2016-01-16 23:24:09
2016-01-16 23:27:41
2016-01-16 23:29:10
2016-01-16 23:30:33
2016-01-16 23:30:58
2016-01-16 23:32:53
2016-01-16 23:37:11
2016-01-16 23:37:26


2016-02-14 23:11:14
2016-02-14 23:12:17
2016-02-14 23:17:07
2016-02-14 23:17:59
2016-02-14 23:18:36
2016-02-14 23:19:31
2016-02-14 23:20:17
2016-02-14 23:20:26
2016-02-14 23:23:18
2016-02-14 23:25:59
2016-02-14 23:27:32
2016-02-14 23:28:09
2016-02-14 23:33:23
2016-02-14 23:33:43
2016-02-14 23:36:41
2016-02-14 23:42:38
2016-02-14 23:43:31
2016-02-14 23:48:22
2016-02-14 23:50:45
2016-02-14 23:53:24
2016-02-14 23:53:31
2016-02-14 23:54:18
2016-02-14 23:54:22
2016-02-14 23:54:51
2016-02-14 23:55:23
2016-02-14 23:55:39
2016-02-14 23:55:59
2016-02-14 23:57:04
2016-02-14 23:57:48
2016-02-14 23:57:56
2016-02-14 23:58:09
2016-02-14 23:58:13
2016-02-14 23:58:19
2016-02-14 23:58:42
2016-02-14 23:58:47
2016-02-14 23:58:49
2016-02-14 23:58:54
2016-02-14 23:59:02
2016-02-14 23:59:19
2016-02-14 23:59:32
2016-02-14 23:59:44
2016-02-15 22:38:34
2016-02-15 22:38:36
2016-02-15 22:38:40
2016-02-15 22:43:25
2016-02-15 22:43:26
2016-02-15 22:44:44
2016-02-15 22:49:05
2016-02-15 23:03:43
2016-02-15 23:03:58


2016-03-17 23:50:31
2016-03-17 23:51:14
2016-03-17 23:51:15
2016-03-17 23:53:49
2016-03-17 23:53:53
2016-03-17 23:53:59
2016-03-17 23:54:21
2016-03-17 23:54:43
2016-03-17 23:55:06
2016-03-17 23:55:13
2016-03-17 23:55:15
2016-03-17 23:55:39
2016-03-17 23:56:07
2016-03-17 23:56:26
2016-03-17 23:56:29
2016-03-17 23:56:30
2016-03-17 23:57:10
2016-03-17 23:57:20
2016-03-17 23:58:13
2016-03-17 23:58:16
2016-03-17 23:58:27
2016-03-17 23:58:55
2016-03-17 23:59:22
2016-03-17 23:59:51
2016-03-18 21:39:33
2016-03-18 21:51:05
2016-03-18 22:04:32
2016-03-18 22:11:13
2016-03-18 22:36:45
2016-03-18 22:52:00
2016-03-18 22:55:30
2016-03-18 23:02:34
2016-03-18 23:07:16
2016-03-18 23:15:19
2016-03-18 23:16:52
2016-03-18 23:17:50
2016-03-18 23:17:51
2016-03-18 23:18:47
2016-03-18 23:24:21
2016-03-18 23:25:10
2016-03-18 23:29:11
2016-03-18 23:29:15
2016-03-18 23:30:33
2016-03-18 23:30:57
2016-03-18 23:31:12
2016-03-18 23:32:24
2016-03-18 23:36:37
2016-03-18 23:37:16
2016-03-18 23:39:24
2016-03-18 23:40:52


2016-04-15 23:55:50
2016-04-15 23:56:21
2016-04-15 23:57:04
2016-04-15 23:57:28
2016-04-15 23:57:32
2016-04-15 23:57:40
2016-04-15 23:57:50
2016-04-15 23:58:00
2016-04-15 23:58:34
2016-04-15 23:58:37
2016-04-15 23:59:53
2016-04-16 19:36:57
2016-04-16 19:37:43
2016-04-16 19:38:07
2016-04-16 19:49:05
2016-04-16 19:55:45
2016-04-16 20:14:29
2016-04-16 20:15:04
2016-04-16 20:16:21
2016-04-16 20:23:28
2016-04-16 20:31:05
2016-04-16 21:32:39
2016-04-16 21:34:48
2016-04-16 21:39:53
2016-04-16 21:49:58
2016-04-16 22:00:11
2016-04-16 22:06:16
2016-04-16 22:16:37
2016-04-16 22:29:55
2016-04-16 22:30:35
2016-04-16 22:33:49
2016-04-16 22:35:07
2016-04-16 22:42:13
2016-04-16 22:49:54
2016-04-16 22:50:32
2016-04-16 22:52:49
2016-04-16 22:54:15
2016-04-16 22:59:53
2016-04-16 23:03:51
2016-04-16 23:17:17
2016-04-16 23:21:28
2016-04-16 23:23:01
2016-04-16 23:27:50
2016-04-16 23:33:32
2016-04-16 23:46:50
2016-04-16 23:54:15
2016-04-16 23:55:10
2016-04-16 23:56:19
2016-04-16 23:57:01
2016-04-16 23:57:02


...

2017-08-01 23:39:03
2017-08-01 23:39:05
2017-08-01 23:40:46
2017-08-01 23:43:03
2017-08-01 23:43:08
2017-08-01 23:44:55
2017-08-01 23:45:41
2017-08-01 23:45:52
2017-08-01 23:46:07
2017-08-01 23:47:35
2017-08-01 23:47:38
2017-08-01 23:48:22
2017-08-01 23:49:36
2017-08-01 23:52:33
2017-08-01 23:56:27
2017-08-01 23:57:06
2017-08-01 23:57:13
2017-08-01 23:57:27
2017-08-01 23:57:44
2017-08-01 23:58:03
2017-08-01 23:58:17
2017-08-01 23:58:23
2017-08-01 23:58:33
2017-08-01 23:58:34
2017-08-01 23:58:45
2017-08-01 23:58:48
2017-08-01 23:58:54
2017-08-01 23:58:57
2017-08-01 23:59:08
2017-08-01 23:59:29
2017-08-01 23:59:33
2017-08-01 23:59:51
2017-08-01 23:59:56
2017-08-03 22:12:11
2017-08-03 22:23:43
2017-08-03 22:28:10
2017-08-03 22:28:25
2017-08-03 22:37:25
2017-08-03 22:40:32
2017-08-03 22:41:49
2017-08-03 22:42:12
2017-08-03 22:55:09
2017-08-03 23:01:56
2017-08-03 23:11:12
2017-08-03 23:14:03
2017-08-03 23:19:47
2017-08-03 23:22:22
2017-08-03 23:23:00
2017-08-03 23:24:35
2017-08-03 23:25:33


2017-09-23 23:34:24
2017-09-23 23:35:49
2017-09-23 23:36:08
2017-09-23 23:37:00
2017-09-23 23:42:45
2017-09-23 23:44:32
2017-09-23 23:45:26
2017-09-23 23:45:48
2017-09-23 23:48:38
2017-09-23 23:49:03
2017-09-23 23:49:06
2017-09-23 23:50:03
2017-09-23 23:50:19
2017-09-23 23:50:24
2017-09-23 23:50:32
2017-09-23 23:51:46
2017-09-23 23:52:04
2017-09-23 23:52:25
2017-09-23 23:52:54
2017-09-23 23:53:12
2017-09-23 23:53:55
2017-09-23 23:54:26
2017-09-23 23:54:56
2017-09-23 23:55:02
2017-09-23 23:55:03
2017-09-23 23:55:09
2017-09-23 23:56:41
2017-09-23 23:56:50
2017-09-23 23:57:55
2017-09-23 23:58:23
2017-09-23 23:59:54
2017-09-25 21:28:43
2017-09-25 21:34:51
2017-09-25 21:42:33
2017-09-25 21:45:42
2017-09-25 21:46:47
2017-09-25 21:48:07
2017-09-25 21:48:36
2017-09-25 21:48:59
2017-09-25 22:08:08
2017-09-25 22:17:09
2017-09-25 22:45:49
2017-09-25 22:48:00
2017-09-25 22:50:55
2017-09-25 22:53:55
2017-09-25 23:03:21
2017-09-25 23:31:22
2017-09-25 23:49:05
2017-09-25 23:49:47
2017-09-25 23:49:54


2017-11-13 20:47:04
2017-11-13 20:47:54
2017-11-13 21:36:12
2017-11-13 21:45:42
2017-11-13 21:47:54
2017-11-13 21:49:25
2017-11-13 21:52:25
2017-11-13 22:05:35
2017-11-13 22:10:41
2017-11-13 22:10:59
2017-11-13 22:11:17
2017-11-13 22:12:55
2017-11-13 22:13:42
2017-11-13 22:42:18
2017-11-13 23:28:54
2017-11-13 23:29:00
2017-11-13 23:30:16
2017-11-13 23:30:27
2017-11-13 23:32:46
2017-11-13 23:36:48
2017-11-13 23:38:16
2017-11-13 23:42:37
2017-11-13 23:44:03
2017-11-13 23:45:28
2017-11-13 23:47:23
2017-11-13 23:48:29
2017-11-13 23:48:36
2017-11-13 23:51:00
2017-11-13 23:51:38
2017-11-13 23:51:46
2017-11-13 23:55:47
2017-11-13 23:56:02
2017-11-13 23:56:27
2017-11-13 23:56:35
2017-11-13 23:56:54
2017-11-13 23:56:57
2017-11-13 23:57:27
2017-11-13 23:57:30
2017-11-13 23:57:31
2017-11-13 23:57:32
2017-11-13 23:57:36
2017-11-13 23:58:04
2017-11-13 23:58:06
2017-11-13 23:58:17
2017-11-13 23:58:48
2017-11-13 23:58:55
2017-11-13 23:59:05
2017-11-13 23:59:21
2017-11-13 23:59:38
2017-11-13 23:59:50


2017-12-27 17:58:19
2017-12-27 18:02:00
2017-12-27 18:03:42
2017-12-27 18:06:00
2017-12-27 18:43:57
2017-12-27 19:15:17
2017-12-27 19:17:45
2017-12-27 19:51:20
2017-12-27 19:53:16
2017-12-27 20:37:20
2017-12-27 21:25:33
2017-12-27 21:50:00
2017-12-27 22:16:39
2017-12-27 22:33:09
2017-12-27 23:18:58
2017-12-27 23:26:20
2017-12-27 23:26:23
2017-12-27 23:27:25
2017-12-27 23:30:15
2017-12-27 23:30:18
2017-12-27 23:31:45
2017-12-27 23:35:40
2017-12-27 23:39:18
2017-12-27 23:43:37
2017-12-27 23:44:28
2017-12-27 23:45:02
2017-12-27 23:48:05
2017-12-27 23:49:20
2017-12-27 23:50:53
2017-12-27 23:51:56
2017-12-27 23:52:00
2017-12-27 23:52:54
2017-12-27 23:53:28
2017-12-27 23:53:55
2017-12-27 23:54:00
2017-12-27 23:54:06
2017-12-27 23:54:52
2017-12-27 23:55:03
2017-12-27 23:55:17
2017-12-27 23:55:27
2017-12-27 23:56:16
2017-12-27 23:57:08
2017-12-27 23:57:11
2017-12-27 23:57:22
2017-12-27 23:58:06
2017-12-27 23:58:45
2017-12-27 23:58:49
2017-12-27 23:58:54
2017-12-27 23:58:56
2017-12-27 23:59:18


2018-02-17 22:57:58
2018-02-17 23:04:36
2018-02-17 23:05:16
2018-02-17 23:05:31
2018-02-17 23:05:50
2018-02-17 23:30:07
2018-02-17 23:37:16
2018-02-17 23:38:17
2018-02-17 23:40:35
2018-02-17 23:54:58
2018-02-18 20:48:58
2018-02-18 20:51:02
2018-02-18 20:54:56
2018-02-18 21:04:04
2018-02-18 21:22:18
2018-02-18 21:30:43
2018-02-18 21:32:25
2018-02-18 21:34:11
2018-02-18 22:02:50
2018-02-18 22:17:54
2018-02-18 22:29:07
2018-02-18 22:31:38
2018-02-18 22:38:11
2018-02-18 22:41:09
2018-02-18 22:42:48
2018-02-18 22:47:09
2018-02-18 22:59:49
2018-02-18 23:03:52
2018-02-18 23:20:19
2018-02-18 23:56:51
2018-02-19 22:17:38
2018-02-19 22:20:48
2018-02-19 22:25:00
2018-02-19 22:31:20
2018-02-19 22:32:32
2018-02-19 22:42:37
2018-02-19 22:52:30
2018-02-19 23:02:49
2018-02-19 23:05:45
2018-02-19 23:07:40
2018-02-19 23:10:37
2018-02-19 23:11:45
2018-02-19 23:14:40
2018-02-19 23:20:05
2018-02-19 23:27:37
2018-02-19 23:38:18
2018-02-19 23:47:41
2018-02-19 23:50:20
2018-02-19 23:52:44
2018-02-19 23:59:48


In [5]:
len(df)

31907

### Vectorize tweets and predict toxicity with model

In [6]:
# Load vectorizers
word_vectorizer = pickle.load(open("models/word_vectorizer.pickle", "rb" ))
char_vectorizer = pickle.load(open("models/char_vectorizer.pickle", "rb" ))

In [7]:
# Vectorize tweets
tweet_word_features = word_vectorizer.transform(df['text'])
tweet_char_features = char_vectorizer.transform(df['text'])
tweet_features = hstack([tweet_word_features, tweet_char_features])

In [8]:
# Load models
loaded_models = {}
for col in target_columns:
    loaded_models[col] = pickle.load(open('models/model_{}.sav'.format(col), "rb" ))

In [9]:
# Run models
for col in target_columns:
    df[col] = loaded_models[col].predict_proba(tweet_features)[:, 1]

In [10]:
df.head()

,date,fullname,id,likes,replies,retweets,text,url,user,month,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,2016-01-31 23:59:44,MissKatsuragi,693946797782204417,0.0,0.0,0.0,Here's how crazy it was to get an abortion bef...,/misskatsuragi/status/693946797782204417,misskatsuragi,2016-01-01,0.001299,0.000174,0.000072,0.000024,0.000311,0.000021
1,2016-01-31 23:59:31,KasaFrackin NoTrump,693946744917331968,0.0,0.0,0.0,"@ShannonBream Kim Quade, âWould a President ...",/KasaFrackin/status/693946744917331968,KasaFrackin,2016-01-01,0.006734,0.000685,0.003013,0.000021,0.000394,0.000026
2,2016-01-31 23:59:01,Patriot,693946617364418560,0.0,0.0,0.0,The abortion of JFKâs children was evil â ...,/jmj4life7/status/693946617364418560,jmj4life7,2016-01-01,0.000627,0.000129,0.000210,0.000014,0.000241,0.000022
3,2016-01-31 23:58:57,Zack Johnson,693946602021601281,2.0,0.0,0.0,How can anyone support abortion?,/Zack_Johnson001/status/693946602021601281,Zack_Johnson001,2016-01-01,0.001206,0.000239,0.000055,0.000033,0.000151,0.000051
4,2016-01-31 23:58:51,Watt Dahale,693946578424430592,0.0,1.0,0.0,@g8r84 bet your pro abortion too! Talk about b...,/WattDahale/status/693946578424430592,WattDahale,2016-01-01,0.010389,0.000370,0.004584,0.000042,0.004399,0.000042


In [11]:
df['classification'] = df[target_columns].idxmax(axis=1)


In [12]:
df['max_value'] = df[target_columns].max(axis=1)
df['likes'] = df['likes'].astype(float)

In [13]:
# export df
export_filename = "classified/FINAL_DATA/abortion"
df.to_csv(export_filename)

In [14]:
df.head()


,date,fullname,id,likes,replies,retweets,text,url,user,month,toxic,severe_toxic,obscene,threat,insult,identity_hate,classification,max_value
0,2016-01-31 23:59:44,MissKatsuragi,693946797782204417,0.0,0.0,0.0,Here's how crazy it was to get an abortion bef...,/misskatsuragi/status/693946797782204417,misskatsuragi,2016-01-01,0.001299,0.000174,0.000072,0.000024,0.000311,0.000021,toxic,0.001299
1,2016-01-31 23:59:31,KasaFrackin NoTrump,693946744917331968,0.0,0.0,0.0,"@ShannonBream Kim Quade, âWould a President ...",/KasaFrackin/status/693946744917331968,KasaFrackin,2016-01-01,0.006734,0.000685,0.003013,0.000021,0.000394,0.000026,toxic,0.006734
2,2016-01-31 23:59:01,Patriot,693946617364418560,0.0,0.0,0.0,The abortion of JFKâs children was evil â ...,/jmj4life7/status/693946617364418560,jmj4life7,2016-01-01,0.000627,0.000129,0.000210,0.000014,0.000241,0.000022,toxic,0.000627
3,2016-01-31 23:58:57,Zack Johnson,693946602021601281,2.0,0.0,0.0,How can anyone support abortion?,/Zack_Johnson001/status/693946602021601281,Zack_Johnson001,2016-01-01,0.001206,0.000239,0.000055,0.000033,0.000151,0.000051,toxic,0.001206
4,2016-01-31 23:58:51,Watt Dahale,693946578424430592,0.0,1.0,0.0,@g8r84 bet your pro abortion too! Talk about b...,/WattDahale/status/693946578424430592,WattDahale,2016-01-01,0.010389,0.000370,0.004584,0.000042,0.004399,0.000042,toxic,0.010389


### Visualizations

In [15]:
# visualize obscenity
#df.groupby('month')['obscene'].mean().plot()

In [16]:
# df['likes'] = df['likes'].astype(float)

# normalizing identity hate by tweet popularity
#df['popular_obscene'] = df['obscene']*df['likes']
#df.groupby('month')['popular_obscene'].mean().plot()

In [17]:
# normalizing identity hate by number of popular tweets
#(df.groupby('month')['popular_obscene'].sum() / df.groupby('month')['likes'].sum()).plot()

In [18]:
# normalizing popular tweets by number of popular tweets
#(df.groupby('month')['popular_obscene'].sum() / df.groupby('month')['likes'].count()).plot()